# Dynamic Programming
Dynamic programming (DP) is not just one algorithm, but is actually a technique for designing algorithms. I.e., by using the dynamic programming technique on appropriate problems you can come up with very powerful algorithms for solving those problems. So the great news is that rather than memorizing 1,000 different algorithms you can learn this technique to derive those algorithms yourself.

In addition, the set of typical problems that DP is good at is quite broad and interesting. Often you can start thinking DP when there is some optimization involved or perhaps some exhaustive search and if the data structures you are searching or optimizing over exhibit some ordering, like left-right or tree or intervals. So what is DP? If I had to describe the technique in one sentence, I would say: DP is describing a recursive solution to the sub-problem and then carefully implement the recursive algorithm solve the problem the most efficient order while storing useful sub-solutions for later use. But the only way people really learn the DP technique is by applying it on multiple problems until you get the hang of it.

## Fibonacci 
Let's take a classic recursive problem as an example, the [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_number). Which can be written by the well known recurrence `F(n) = F(n-1) + F(n-2)` where `F(0) = 0 and F(1) = 1`.  As you see in the diagram, calculating the 5th Fibonacci number can be represented as a tree of sub-problems, where the node would represent the answer of `F(i)` and the arrows represent the dependency of other parts of the sequence. 
![Fibonacci Computation](fibonacci_5.png)

The tree visualization helps us infer two important properties:
- There is a lot of repetitive computation (identical sub-trees).
- The actual necessary computations can be seen by following the left most edge from the root `F(5)` to the lowest left descendent `F(0)` or the height of the tree or 6 nodes.

So from this tree visualization we can infer that the actual required computations is $O(N)$, but if we blindly computed the recursive algorithm above we would do $O(2^N)$ because of all of the repetition. So this is exactly the kind of problem DP can help us with. I like to thing of two main approach's to DP design, 1) Top Down or Recursion + Memoization and 2) Bottom Up or DP Table design. Since, this problem is given as a recursive algorithm let's start with Top Down.

### Top Down Dynamic Programming
[Memoization](https://en.wikipedia.org/wiki/Memoization) simply means we will store computations we make so that if we ever need them again we will just look them up rather than recomputing. That sounds like a great idea for calculating our Fibonacci numbers and dealing with the repetitive calculations. So let's first take a look at the brute-force recursive algorithm, with a little work counter added for illustration.


In [11]:
class Fib(object):
    def __init__(self):
        self.work = 0
    def results(self, n):
        return f"The {n}th Fibonacci number is {self.get(n)} and requires {self.work} computations" 
    def get(self, n):
        self.work += 1
        if n == 0:
            return 0
        if n == 1:
            return 1
        return self.get(n-1) + self.get(n-2)

print(Fib().results(6))

The 6th Fibonacci number is 8 and requires 25 computations


Applying memoization is as simple as it sounds, we just need to store past calculations and look them up before blindly computing. The common storage mechanism is a hash table.

In [15]:
class FibMemo(Fib):
    def __init__(self):
        self.cache = {}
    def get(self, n):
        if n in self.cache:
            return self.cache[n]
        f = super().get(n)
        self.cache[n] = f
        return f
    
print(FibMemo().results(6))
    

The 6th Fibonacci number is 8 and requires 7 computations


At its core, thats really all there is to it. Once you have a recursive algorithm that exhibits the computation explosion because or repetition you can really improve performance by saving computation and adding storage. Similar to other recursive algorithms, however, there can be some benefit to transforming the problem into an iterative algorithm. When you do this you will often benefit by making the algorithm even more efficient by using storage more effectively, both in terms of the function call stack and auxillary storage for the problem itself. In terms of dynamic programming, we consider this the Bottom Up Technique or DP Table.

### Bottom Up Dynamic Programming
To motivate the bottom-up technique, let's take a different visualization of the fibonacci number computations, by flattening the left edge of the tree we get the following directed acyclic graph (DAG) structure ![Fibonacci DAG](fibonacci_5_linear.png).

The top-down approach worked from the right of the DAG towards the left or from the final desired solution through the dependencies until reaching the base cases. The bottom-up technique will move from the left of the DAG to the right or start with the bases cases and slowly build information until we reach the final solution.

In [21]:
class FibBottomUp(Fib):
    def __init__(self):
        self.cache = {}
    def get(self, n):
        c = self.cache
        c[0] = 0
        c[1] = 1
        self.work = 2
        for i in range(2,n+1):
            self.work += 1
            c[i] = c[i-1] + c[i-2]
        return c[n]
print(FibBottomUp().results(6))

The 6th Fibonacci number is 8 and requires 7 computations


So now we've achieved the same computation time saving, i.e., going from $O(2^N)$ brute-force to $O(N)$, but we are also saving storage by not recursively calling $N$ functions to calculate the number. However, as is common with the bottom-up formulation, in this case we can find more storage saving by understanding how we are traversing the DAG. Since we are traversing from left to right and the dependency of any node is at most two nodes back we realize that we only need to store 2 values, $O(1)$, rather than retaining all $O(N)$ previous numbers.

In [30]:
class FibNoCache(Fib):
    def get(self, n):
        if n == 0:
            return 0
        if n == 1:
            return 1
        self.work = 2
        pp = 0
        p = 1
        for i in range(2, n+1):
            self.work += 1
            f = p + pp
            pp = p
            p = f
        return p
print(FibNoCache().results(6))

The 6th Fibonacci number is 8 and requires 7 computations
